In [ ]:
!unzip -qq '/content/drive/MyDrive/Feature_Engineering/EMA_우울증예측/EMA_우울증예측.zip'

replace StduentLife_EMA/EMA/EMA_definition.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
from glob import glob
from tqdm import tqdm
import json

from matplotlib import pyplot as plt

- 우울증 유/무 와 user id가 담긴 train/test csv 파일 로드하기
- train/test csv파일에서 user id 파싱하기

In [ ]:
train = pd.read_csv('train_Y.csv')
submit = pd.read_csv('sample_submit.csv')

In [ ]:
train_user = train['user']
test_user = submit['user']
y_train = train['label'].to_numpy()

In [ ]:
total_user = pd.concat([train_user, test_user])
total_user = total_user.sort_values().reset_index(drop=True)

total_user[:5]

0    u00
1    u01
2    u02
3    u03
4    u04
Name: user, dtype: object

- Sleep, Social, Activity와 관련된 EMA 데이터 불러오기
- 이때, user id에 대응되는 EMA 데이터만 로드한다.

glob의 사용법 <br>
 : 이름과 일치하는 모든 파일의 리스트를 반환
```python
from golb import golb
golb('*.exe') # 현재 디렉터리의 .exe파일 모두 리스트형태로 저장
```

In [ ]:
sleep = sorted(glob('StduentLife_EMA/EMA/response/Sleep/*.json'))
social = sorted(glob('StduentLife_EMA/EMA/response/Social/*.json'))
activity = sorted(glob('StduentLife_EMA/EMA/response/Activity/*.json'))

#### Sleep json 파일의 구조
- hour : 시간
  - 1 - 19 : 3시간 미만 부터 30분 단위로 12시간 까지
- location : GPS 좌표
- rate : 학생들의 수면 평가
  - [1]Very good, [2]Fairly good, [3]Fairly bad, [4]Very bad
- social : 깨어있는데 몇 번이나 어려웠는지 (교실 or 밥 먹으면서 소셜 활동에 참여하면서)
  - [1]None, [2]Once, [3]Twice, [4]Three or more times
- resp_time : unix time

user_id에 대응대는 json파일이 있으면 저장하려고 함 <br>
- json 파일 명 `StduentLife_EMA/EMA/response/Sleep/Sleep_u00.json`
기존 sleep 변수에는 json파일명이 문자열 형태로 저장되어 있음


In [ ]:
tmp = []

# sleep 관련
for i in range(len(sleep)) :
  for k in range(len(total_user)) : 
    if total_user[k] in sleep[i] :
      tmp.append(sleep[i])

sleep = np.array(tmp)

46

#### Social json 파일의 구조
- location : GPS
- number : 어제 접촉한 사람 수
  - [1]0-4 persons, [2]5-9 persons, [3]10-19 persons, [4]20-49 persons, [5]50-99 persons, [6]over 100 persons
- resp_time : unix time


In [ ]:
tmp = []

# social 관련
for i in range(len(social)) :
  for k in range(len(total_user)) : 
    if total_user[k] in social[i] :
      tmp.append(social[i])

social = np.array(tmp)

#### Activity json 파일의 구조
- other_relaxing : 다른 사람과 쉴 때 
  - [1]0-10%, [2]11-25%, [3]26-50%, [4]51-75%, [5]76-100%
- other_working : 다른 사람과 일할 때
  - [1]0-10%, [2]11-25%, [3]26-50%, [4]51-75%, [5]76-100%
- relaxing : 혼자 쉴 때 
  - [1]0-10%, [2]11-25%, [3]26-50%, [4]51-75%, [5]76-100%
- resp_time : unix time
- working : 혼자 일할 때
  - [1]0-10%, [2]11-25%, [3]26-50%, [4]51-75%, [5]76-100%
- location : GPS

In [ ]:
tmp = []

for i in range(len(activity)) :
  for k in range(len(total_user)) : 
    if total_user[k] in activity[i] :
      tmp.append(activity[i])

activity = np.array(tmp)

# 1-1 EMA데이터 파싱 - Sleep  

목표: 아래 설문항목에 응답한 user의 EAM 결과를 적절한 인덱스에 대응되는 데이터를 파싱한다.

```
[input]
--------------
- json : User의 EMA 응답 결과(Index)가 담긴 json파일

[output]
--------------
- raw data : 대응되는 Index의 값을 추출한 데이터
    
```

In [ ]:
################################################################

# 1. json load를 통해서 sleep에 저장된 파일을 하나씩 로드
# 2. 유저의 응답 중 'question_id'가 'hour','rate'인 데이터를 파싱
# 3. 이때, 각 응답은 인덱스 값이므로 해당 인덱스에 대응되는 실제 값으로 변환

# How many hours did you sleep last night?
# [1] -> 3, [2]->3.5, [3]->4.0, [4]->4.5 , ... , [18]->11.5, [19]->12

# Sleep_rate
# [1]Very good -> 4, [2]Fairly good -> 3, [3]Fairly bad -> 2, [4]Very bad ->1

# 만약 대응되는 인덱스가 없거나 대응되는 답변이 없다면 해당 데이터는 'NaN'으로 설정한다.

################################################################

people_hour = []
people_rate = []

for sp in tqdm(sleep) : 
    
  person_hour = []
  person_rate = []

  # json load를 통해서 sleep에 저장된 파일을 하나씩 로드
  with open(sp) as json_file:
    sleep_data = json.load(json_file)
  for res in sleep_data :

    if 'hour' in res :
      # 유저의 응답 중 'question_id'가 'hour'인 데이터 파싱
      hour = {'1':3,'2':3.5,'3':4,'4':4.5,'5':5,'6':5.5,'7':6,'8':6.5,'9':7,'10':7.5,'11':8,'12':8.5,'13':9,'14':9.5,'15':10,'16':10.5,'17':11,'18':11.5,'19':12}

      if res['hour'] in hour: 
        person_hour.append(hour[res['hour']])
      else:
        person_hour.append('NaN')

    if 'rate' in res :
      # 유저의 응답 중 'question_id'가 'rate'인 데이터 파싱
      rate = {'1':4, '2':3, '3':2, '4':1} 
      
      if res['rate'] in rate:
        person_rate.append(rate[res['rate']])
      else:
        person_rate.append('NaN')

  people_hour.append(person_hour)
  people_rate.append(person_rate)

100%|██████████| 46/46 [00:00<00:00, 4260.34it/s]


# EMA데이터 파싱 - Social  

목표: 아래 설문항목에 응답한 user의 EAM 결과를 적절한 인덱스에 대응되는 데이터를 파싱한다.

```
[input]
--------------
- json : User의 EMA 응답 결과(Index)가 담긴 json파일

[output]
--------------
- raw data : 대응되는 Index의 값을 추출한 데이터
    
```

In [ ]:
################################################################

# 1. json load를 통해서 sleep에 저장된 파일을 하나씩 로드한다.
# 2. 유저의 응답 중 'question_id'가 'number'인 데이터를 파싱한다.
# 3. 이때, 각 응답은 인덱스 값이므로 해당 인덱스에 대응되는 실제 값으로 변환한다.

# Social
# [1]->0, [2]->5, [3]->10, [4]->20, [5]->50, [6]->100

# 만약 대응되는 인덱스가 없거나 대응되는 답변이 없다면 해당 데이터는 'NaN'으로 설정한다.

################################################################

people_contact = list() 

for soc in tqdm(social) :
  person_contact = []

  # json load를 통해서 sleep에 저장된 파일을 하나씩 로드한다.
  with open(soc) as json_file:
    social_data = json.load(json_file)

  for res in social_data :
    if 'number' in res :
      # 유저의 응답 중 'question_id'가 'number'인 데이터를 파싱한다. 
      number = {'1':0, '2':5, '3':10, '4':20, '5':50, '6':100} 
      
      if res['number'] in number:
        person_contact.append(number[res['number']])
      else:
        person_contact.append('NaN')

  people_contact.append(person_contact)

100%|██████████| 46/46 [00:00<00:00, 6220.19it/s]


# 1-3 EMA데이터 파싱 - Activity  

목표: 아래 설문항목에 응답한 user의 EAM 결과를 적절한 인덱스에 대응되는 데이터를 파싱한다.

```
[input]
--------------
- json : User의 EMA 응답 결과(Index)가 담긴 json파일

[output]
--------------
- raw data : 대응되는 Index의 값을 추출한 데이터
    
```

In [ ]:
################################################################

# 1. json load를 통해서 sleep에 저장된 파일을 하나씩 로드한다.
# 2. 유저의 응답 중 'question_id'가 'other_relaxing','other_working',
#                                 'relaxing','working'인 데이터를 파싱한다.
# 3. 이때, 각 응답은 인덱스 값이므로 해당 인덱스에 대응되는 실제 값으로 변환한다.

# Activity
# [1]->0, [2]->0.11, [3]->0.26, [4]->0.51, [5]->0.76

# 만약 대응되는 인덱스가 없거나 대응되는 답변이 없다면 해당 데이터는 'NaN'으로 설정한다.

################################################################

people_work_alone = [] 
people_work_other = []
people_relaxing_alone = []
people_relaxing_other = []

for act in tqdm(activity) :

  person_work_alone = []
  person_work_other = []
  person_relaxing_alone = [] 
  person_relaxing_other = []

  # json load를 통해서 sleep에 저장된 파일을 하나씩 로드한다.
  with open(act) as json_file:
    activity_data = json.load(json_file)

  for res in activity_data :

    act = {'1':0, '2':0.11, '3':0.26, '4':0.51, '5':0.76}

    # 유저의 응답 중 'question_id'가 'other_relaxing'인 데이터를 파싱한다.
    if 'other_relaxing' in res :
      if res['other_relaxing'] in number:
        person_contact.append(act[res['other_relaxing']])
      else:
        person_contact.append('NaN')

    # 유저의 응답 중 'question_id'가 'other_working'인 데이터를 파싱한다.
    if 'other_working' in res :
      if res['other_working'] in number:
        person_contact.append(act[res['other_working']])
      else:
        person_contact.append('NaN')

    # 유저의 응답 중 'question_id'가 'relaxing'인 데이터를 파싱한다.
    if 'relaxing' in res :
      if res['relaxing'] in number:
        person_contact.append(act[res['relaxing']])
      else:
        person_contact.append('NaN')

    # 유저의 응답 중 'question_id'가 'working'인 데이터를 파싱한다.
    if 'working' in res :

      if res['working'] in number:
        person_contact.append(act[res['working']])
      else:
        person_contact.append('NaN')

  people_relaxing_other.append(person_relaxing_other)
  people_work_other.append(person_work_other)
  people_relaxing_alone.append(person_relaxing_alone)
  people_work_alone.append(person_work_alone)

100%|██████████| 46/46 [00:00<00:00, 5235.20it/s]


- 앞서 알맞게 데이터를 파싱했다면 아래 부분은 해당 데이터를 기반으로 Feature를 추출하기 위한 준비단계이다.
- 가장먼저 데이터의 type을 float형으로 변경하고, 판다스의 DataFrame으로 변경한다. 그리고 train/test 데이터셋 파싱을 위해서 가장 첫번째 column에 user id를 추가한다.

In [ ]:
student_sleep = pd.DataFrame(people_hour).astype('float64')
student_sleep_rate = pd.DataFrame(people_rate).astype('float64')
student_contact = pd.DataFrame(people_contact).astype('float64')
student_working_alone = pd.DataFrame(people_work_alone).astype('float64')
student_working_other = pd.DataFrame(people_work_other).astype('float64')
student_relaxing_alone = pd.DataFrame(people_relaxing_alone).astype('float64')
student_relaxing_other = pd.DataFrame(people_relaxing_other).astype('float64')

student_sleep[:5]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
0,6.5,5.0,6.5,6.0,6.5,6.5,5.5,6.0,4.5,6.5,4.5,6.5,7.0,4.0,7.5,4.5,7.0,7.0,5.0,4.5,5.5,5.0,6.5,4.0,4.0,9.5,7.0,7.0,4.0,4.0,6.5,6.5,7.0,5.5,4.0,5.0,4.0,4.0,4.0,8.0,10.0,7.5,7.5,7.0,5.5,7.5,7.0,7.0,7.0,6.0,8.5,7.0,3.0,7.5,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.5,6.0,6.0,6.0,7.5,6.0,6.5,6.5,6.5,6.5,6.0,6.5,7.0,5.5,6.5,5.0,6.0,5.5,5.0,7.0,5.0,6.5,6.5,6.0,6.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.5,6.0,6.5,6.0,6.5,6.0,6.0,6.5,6.5,6.5,6.5,6.0,7.0,6.0,6.0,6.5,6.5,7.0,6.5,6.5,6.0,5.5,6.5,7.0,8.5,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.0,6.5,7.0,6.5,6.5,6.0,7.5,6.0,6.0,6.0,5.5,7.5,4.5,5.5,6.0,6.5,6.0,7.5,7.0,6.5,6.5,6.0,6.0,6.0,6.0,9.0,10.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.5,5.5,5.5,5.5,6.0,5.5,5.5,5.5,6.0,6.0,5.5,6.5,6.5,6.0,6.0,5.0,5.5,6.0,5.0,6.5,5.5,6.5,6.0,5.5,5.5,5.5,5.5,5.5,5.5,6.0,6.5,5.5,6.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
student_sleep['uid'] = total_user
student_sleep_rate['uid'] = total_user
student_contact['uid'] = total_user
student_working_alone['uid'] = total_user
student_working_other['uid'] = total_user
student_relaxing_alone['uid'] = total_user
student_relaxing_other['uid'] = total_user

student_sleep[:5]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,uid
0,6.5,5.0,6.5,6.0,6.5,6.5,5.5,6.0,4.5,6.5,4.5,6.5,7.0,4.0,7.5,4.5,7.0,7.0,5.0,4.5,5.5,5.0,6.5,4.0,4.0,9.5,7.0,7.0,4.0,4.0,6.5,6.5,7.0,5.5,4.0,5.0,4.0,4.0,4.0,8.0,10.0,7.5,7.5,7.0,5.5,7.5,7.0,7.0,7.0,6.0,8.5,7.0,3.0,7.5,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,u00
1,5.5,6.0,6.0,6.0,7.5,6.0,6.5,6.5,6.5,6.5,6.0,6.5,7.0,5.5,6.5,5.0,6.0,5.5,5.0,7.0,5.0,6.5,6.5,6.0,6.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,u01
2,6.5,6.0,6.5,6.0,6.5,6.0,6.0,6.5,6.5,6.5,6.5,6.0,7.0,6.0,6.0,6.5,6.5,7.0,6.5,6.5,6.0,5.5,6.5,7.0,8.5,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,u02
3,6.0,6.5,7.0,6.5,6.5,6.0,7.5,6.0,6.0,6.0,5.5,7.5,4.5,5.5,6.0,6.5,6.0,7.5,7.0,6.5,6.5,6.0,6.0,6.0,6.0,9.0,10.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,u03
4,5.5,5.5,5.5,5.5,6.0,5.5,5.5,5.5,6.0,6.0,5.5,6.5,6.5,6.0,6.0,5.0,5.5,6.0,5.0,6.5,5.5,6.5,6.0,5.5,5.5,5.5,5.5,5.5,5.5,6.0,6.5,5.5,6.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,u04


In [ ]:
student_sleep = student_sleep[(['uid']+list(range(0,student_sleep.shape[1]-1)))]
student_sleep_rate = student_sleep_rate[(['uid']+list(range(0,student_sleep_rate.shape[1]-1)))]
student_contact = student_contact[(['uid']+list(range(0,student_contact.shape[1]-1)))]
student_working_alone = student_working_alone[(['uid']+list(range(0,student_working_alone.shape[1]-1)))]
student_working_other = student_working_other[(['uid']+list(range(0,student_working_other.shape[1]-1)))]
student_relaxing_alone = student_relaxing_alone[(['uid']+list(range(0,student_relaxing_alone.shape[1]-1)))]
student_relaxing_other = student_relaxing_other[(['uid']+list(range(0,student_relaxing_other.shape[1]-1)))]

student_sleep[:5]

,uid,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
0,u00,6.5,5.0,6.5,6.0,6.5,6.5,5.5,6.0,4.5,6.5,4.5,6.5,7.0,4.0,7.5,4.5,7.0,7.0,5.0,4.5,5.5,5.0,6.5,4.0,4.0,9.5,7.0,7.0,4.0,4.0,6.5,6.5,7.0,5.5,4.0,5.0,4.0,4.0,4.0,8.0,10.0,7.5,7.5,7.0,5.5,7.5,7.0,7.0,7.0,6.0,8.5,7.0,3.0,7.5,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,u01,5.5,6.0,6.0,6.0,7.5,6.0,6.5,6.5,6.5,6.5,6.0,6.5,7.0,5.5,6.5,5.0,6.0,5.5,5.0,7.0,5.0,6.5,6.5,6.0,6.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,u02,6.5,6.0,6.5,6.0,6.5,6.0,6.0,6.5,6.5,6.5,6.5,6.0,7.0,6.0,6.0,6.5,6.5,7.0,6.5,6.5,6.0,5.5,6.5,7.0,8.5,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,u03,6.0,6.5,7.0,6.5,6.5,6.0,7.5,6.0,6.0,6.0,5.5,7.5,4.5,5.5,6.0,6.5,6.0,7.5,7.0,6.5,6.5,6.0,6.0,6.0,6.0,9.0,10.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,u04,5.5,5.5,5.5,5.5,6.0,5.5,5.5,5.5,6.0,6.0,5.5,6.5,6.5,6.0,6.0,5.0,5.5,6.0,5.0,6.5,5.5,6.5,6.0,5.5,5.5,5.5,5.5,5.5,5.5,6.0,6.5,5.5,6.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2-1 Feature extract

목표: 앞서 파싱한 EMA 응답 데이터에서 통계적 feature를 추출한다. 

- 앞서 추가한 ueser id(uid)를 기준으로 trian/test 유저에 데이터를 나눈다. 그리고 해당 유저의 데이터를 pandas에서 제공하는 describe 함수를 이용해 통계적 feature를 추출한다. 
- 통계적 feature는 총 7차원이며 각 구성은 아래 추가설명과 같다. (describe함수의 가장 첫번째 column인 count는 제거한다.)

In [ ]:
# train Data에 대한 feature를 추출

################################################################

# 1. train_user에 해당하는 데이터만 파싱한다. 
# 2. pandas의 describe 함수를 이용해 feature를 추출한다.
    
################################################################

train_features_student_sleep = student_sleep[student_sleep['uid'].isin(train_user)].iloc[:,1:].T.describe().to_numpy()
train_features_student_sleep_rate = student_sleep_rate[student_sleep_rate['uid'].isin(train_user)].iloc[:,1:].T.describe().to_numpy()
train_features_student_contact = student_contact[student_contact['uid'].isin(train_user)].iloc[:,1:].T.describe().to_numpy()
train_features_student_working_alone = student_working_alone[student_working_alone['uid'].isin(train_user)].iloc[:,1:].T.describe().to_numpy()
train_features_student_working_other = student_working_other[student_working_other['uid'].isin(train_user)].iloc[:,1:].T.describe().to_numpy()
train_features_student_relaxing_alone = student_relaxing_alone[student_relaxing_alone['uid'].isin(train_user)].iloc[:,1:].T.describe().to_numpy()
train_features_student_relaxing_other = student_relaxing_other[student_relaxing_other['uid'].isin(train_user)].iloc[:,1:].T.describe().to_numpy()

In [ ]:
# test셋에 대한 feature를 추출

################################################################

# 1. test_user에 해당하는 데이터만 파싱한다. 
# 2. pandas의 describe 함수를 이용해 feature를 추출한다.
    
################################################################

test_features_student_sleep = student_sleep[student_sleep['uid'].isin(test_user)].iloc[:,1:].T.describe().to_numpy()
test_features_student_sleep_rate = student_sleep_rate[student_sleep_rate['uid'].isin(test_user)].iloc[:,1:].T.describe().to_numpy()
test_features_student_contact = student_contact[student_contact['uid'].isin(test_user)].iloc[:,1:].T.describe().to_numpy()
test_features_student_working_alone = student_working_alone[student_working_alone['uid'].isin(test_user)].iloc[:,1:].T.describe().to_numpy()
test_features_student_working_other = student_working_other[student_working_other['uid'].isin(test_user)].iloc[:,1:].T.describe().to_numpy()
test_features_student_relaxing_alone = student_relaxing_alone[student_relaxing_alone['uid'].isin(test_user)].iloc[:,1:].T.describe().to_numpy()
test_features_student_relaxing_other = student_relaxing_other[student_relaxing_other['uid'].isin(test_user)].iloc[:,1:].T.describe().to_numpy()

# 2-2 Feature에서 Nan값 처리하기

- 데이터에 NaN값을 Imputer를 통해서 채운다. 하지만 우울증의 경우 소수의 데이터를 제외한 나머지를 NaN으로 채우면 실제 일부 우울증 패턴이 없어질 수 있다. 따라서 describe를 통해 추출한 Feature에 대해서 Nan에 값이 있을 경우에 Imputer를 통해 NaN을 채운다.

In [ ]:
################################################################
# Imputer를 이용해 Nan 데이터 평균값으로 채우기
################################################################

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# train_features_student_sleep
imputer.fit(train_features_student_sleep)
train_features_student_sleep = imputer.transform(train_features_student_sleep)
test_features_student_sleep = imputer.transform(test_features_student_sleep)

# train_features_student_sleep_rate
imputer.fit(train_features_student_sleep_rate)
train_features_student_sleep_rate = imputer.transform(train_features_student_sleep_rate)
test_features_student_sleep_rate = imputer.transform(test_features_student_sleep_rate)

# train_features_student_contact
imputer.fit(train_features_student_contact)
train_features_student_contact = imputer.transform(train_features_student_contact)
test_features_student_contact = imputer.transform(test_features_student_contact)

# train_features_student_working_alone
imputer.fit(train_features_student_working_alone)
train_features_student_working_alone = imputer.transform(train_features_student_working_alone)
test_features_student_working_alone = imputer.transform(test_features_student_working_alone)

# train_features_student_working_other
imputer.fit(train_features_student_working_other)
train_features_student_working_other = imputer.transform(train_features_student_working_other)
test_features_student_working_other = imputer.transform(test_features_student_working_other)

# train_features_student_relaxing_alone
imputer.fit(train_features_student_relaxing_alone)
train_features_student_relaxing_alone = imputer.transform(train_features_student_relaxing_alone)
test_features_student_relaxing_alone = imputer.transform(test_features_student_relaxing_alone)

# train_features_student_relaxing_other
imputer.fit(train_features_student_relaxing_other)
train_features_student_relaxing_other = imputer.transform(train_features_student_relaxing_other)
test_features_student_relaxing_other = imputer.transform(test_features_student_relaxing_other)

# 3-1 Sleep feature만 이용하여 우울증 예측하기

- 앞에서 구축한 Feature 중 sleep과 관련된 feature만을 사용하여 우울증을 예측한다.
- 분류에는 SVM을 사용하여 주어진 파라미터를 이용하여 분류를 수행한다.

In [ ]:
# Sleep feature만 이용해 우울증을 예측한다. 

#######################################################################

# 1. describe에서 제공하는 통계적 feature 중 count는 제외한 나머지를 사용한다. 

  # train_features_student_sleep
  # train_features_student_sleep_rate
  
#######################################################################

X_train = []

for i in range(train_features_student_sleep.shape[1]) : 
  X_train.append(train_features_student_sleep[1:8,i])
  X_train.append(train_features_student_sleep_rate[1:8,i])


X_train = np.array(X_train)
X_train = X_train.reshape(train_features_student_sleep.shape[1],-1)

X_test = []

for i in range(test_features_student_sleep.shape[1]) : 
    X_test.append(test_features_student_sleep[1:8,i])
    X_test.append(test_features_student_sleep[1:8,i])


X_test = np.array(X_test)
X_test = X_test.reshape(test_features_student_sleep.shape[1],-1)

In [ ]:
# SVM을 이용해 이진분류 수행하기 

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

model = GridSearchCV(SVC(), tuned_parameters)
model.fit(X_train, y_train.reshape(-1))
y_pred = model.predict(X_test)

submit['label'] = y_pred
submit.to_csv("submit_SVC_sleep_features.csv", index=False)

# 3-2 Social feature만 이용하여 우울증 예측하기

- 앞서 수면과 관련된 EMA 데이터를 이용해 우울증을 에측해 성능을 확인했다면, 이제는 Social관련 feature를 이용해 우울증 예측을 진행한다.
- 분류에는 SVM을 사용하며 주어진 파라미터를 이용하여 분류를 수행한다.

In [ ]:
# Social feature만 이용해 우울증을 예측한다. 

#######################################################################

# 1. describe에서 제공하는 통계적 feature 중 count는 제외한 나머지를 사용한다. 
  # train_features_student_contact
    
#######################################################################

X_train = []

for i in range(train_features_student_contact.shape[1]) : 
  X_train.append(train_features_student_contact[1:8,i])


X_train = np.array(X_train)
X_train = X_train.reshape(train_features_student_contact.shape[1],-1)


X_test = []

for i in range(test_features_student_contact.shape[1]) : 
  X_test.append(test_features_student_contact[1:8,i])


X_test = np.array(X_test)
X_test = X_test.reshape(test_features_student_contact.shape[1],-1)

In [ ]:
# SVM을 이용해 이진분류 수행하기 

tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

model = GridSearchCV(SVC(), tuned_parameters)
model.fit(X_train, y_train.reshape(-1))
y_pred = model.predict(X_test)

submit['label'] = y_pred
submit.to_csv("/kaggle/working/submit_SVC_social_features.csv", index=False)

# 3-3 Activity feature만 이용하여 우울증 예측하기

- 앞서 수면, 활동과 관련된 EMA 데이터를 이용해 우울증을 에측해 성능을 확인했다면, 이제는 Activiy와 관련 feature를 이용해 우울증 예측을 진행한다.
- 분류에는 SVM을 사용하며 주어진 파라미터를 이용하여 분류를 수행한다.

In [ ]:
# Activity feature만 이용해 우울증을 예측한다. 

#######################################################################

# 1. describe에서 제공하는 통계적 feature 중 count는 제외한 나머지를 사용한다. 
  # train_features_student_working_alone
  # train_features_student_working_other
  # train_features_student_relaxing_alone
  # train_features_student_relaxing_other
  
#######################################################################

X_train = []

for i in range(train_features_student_sleep.shape[1]) : 
  
  X_train.append(train_features_student_working_alone[1:8,i])
  X_train.append(train_features_student_working_other[1:8,i])
  X_train.append(train_features_student_relaxing_alone[1:8,i])
  X_train.append(train_features_student_relaxing_other[1:8,i])


X_train = np.array(X_train)
X_train = X_train.reshape(train_features_student_sleep.shape[1],-1)

X_test = []

for i in range(test_features_student_sleep.shape[1]) : 
  
  X_test.append(test_features_student_working_alone[1:8,i])
  X_test.append(test_features_student_working_other[1:8,i])
  X_test.append(test_features_student_relaxing_alone[1:8,i])
  X_test.append(test_features_student_relaxing_other[1:8,i])

X_test = np.array(X_test)
X_test = X_test.reshape(test_features_student_sleep.shape[1],-1)

In [ ]:
# SVM을 이용해 이진분류 수행하기 

tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

model = GridSearchCV(SVC(), tuned_parameters)
model.fit(X_train, y_train.reshape(-1))
y_pred = model.predict(X_test)

submit['label'] = y_pred
submit.to_csv("submit_SVC_Activity_features.csv", index=False)

# 3-4 모든 feature를 이용하여 우울증 예측하기

- 앞서 수면, 활동, 사회성과 관련된 EMA 데이터에서 추출한 feature 각각을 이용해 우울증을 분류하였다면, 모든 feautre를 통합하여 분류를 수행한다.
- 분류에는 SVM을 사용하며 주어진 파라미터를 이용하여 분류를 수행한다.

In [ ]:
# Social feature만 이용해 우울증을 예측한다. 

#######################################################################

# 1. describe에서 제공하는 통계적 feature 중 count는 제외한 나머지를 사용한다. 

  # train
  # train_features_student_sleep
  # train_features_student_sleep_rate
  # train_features_student_contact
  # train_features_student_working_alone
  # train_features_student_working_other
  # train_features_student_relaxing_alone
  # train_features_student_relaxing_other
    
#######################################################################

X_train = list()

for i in range(train_features_student_working_alone.shape[1]) : 

  X_train.append(train_features_student_sleep[1:8,i])
  X_train.append(train_features_student_sleep_rate[1:8,i])
  X_train.append(train_features_student_contact[1:8,i])
  X_train.append(train_features_student_working_alone[1:8,i])
  X_train.append(train_features_student_working_other[1:8,i])
  X_train.append(train_features_student_relaxing_alone[1:8,i])
  X_train.append(train_features_student_relaxing_other[1:8,i])

X_train = np.array(X_train)
X_train = X_train.reshape(train_features_student_working_alone.shape[1],-1)

X_test = list()

for i in range(test_features_student_working_alone.shape[1]) : 
  
  X_test.append(test_features_student_sleep[1:8,i])
  X_test.append(test_features_student_sleep_rate[1:8,i])
  X_test.append(test_features_student_contact[1:8,i])
  X_test.append(test_features_student_working_alone[1:8,i])
  X_test.append(test_features_student_working_other[1:8,i])
  X_test.append(test_features_student_relaxing_alone[1:8,i])
  X_test.append(test_features_student_relaxing_other[1:8,i])

X_test = np.array(X_test)
X_test = X_test.reshape(test_features_student_working_alone.shape[1],-1)

In [ ]:
# SVM을 이용해 이진분류 수행하기 

tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

model = GridSearchCV(SVC(), tuned_parameters)
model.fit(X_train, y_train.reshape(-1))
y_pred = model.predict(X_test)

submit['label'] = y_pred
submit.to_csv("submit_SVC_All_features.csv", index=False)

# 결론

- 다양한 EMA 데이터를 통해 추출한 Feature를 융합해서 사용하면 더 좋은 성능으로 우울증 분류가 가능함을 확인할 수 있다. 이를 통해서 실제 Feature representation에 따른 성능차이를 확인할 수 있다.

## 추가실험

- 해당 데이터셋은 데이터 imbalnce 문제가 심각하다. 따라서 이를 해결하기 위한 방법을 추가적으로 사용하여 성능을 향상시킬 수 있다.
- 현재 스켈레톤 코드에서는 사용하지 않는 'Stress', 'Exercise'에 대한 EMA 데이터도 제공한다. 이를 추가적으로 이용하여 성능을 향상시킬 수 있다.

